In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

In [2]:
list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://www.standardbrand.com/en/ecatalog?part...
1,2,https://www.standardbrand.com/en/ecatalog?part...


In [3]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)

list_dict_correct, page = [], 0
while True:

    count_retry = 0
    while True:
        count_retry += 1
        print(page + 1, count_retry)
        try:
            url_request = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start={page * 96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                list_dict = resp.json()

                if list_dict:
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = = = = = = = = = = = = = =

    if page == 0:
        count = list_dict[0]['maxRows']

    # = = = = = = = = = = = = = = =

    list_dict_correct.extend([{'No': dict_['rowId'],
                               'Part Number': dict_['basePart'],
                               'Url': f'''https://www.standardbrand.com/en/ecatalog?part={dict_['webBase'].strip()}&type=p&search=s'''} for dict_ in list_dict])

    # = = = = = = = = = = = = = = =

    print(f' - {len(list_dict_correct)}/{count}')

    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if len(list_dict_correct) == count:
        break

list_dict_correct

1 1
 - 96/116
2 1
 - 116/116


[{'No': 1,
  'Part Number': 'CCR-3',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=CCR3&type=p&search=s'},
 {'No': 2,
  'Part Number': 'CCR-6',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=CCR6&type=p&search=s'},
 {'No': 3,
  'Part Number': 'CCR-7',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=CCR7&type=p&search=s'},
 {'No': 4,
  'Part Number': 'DS-2282',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=DS2282&type=p&search=s'},
 {'No': 5,
  'Part Number': 'LS-312',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=LS312&type=p&search=s'},
 {'No': 6,
  'Part Number': 'NS-35',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=NS35&type=p&search=s'},
 {'No': 7,
  'Part Number': 'NS-56',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=NS56&type=p&search=s'},
 {'No': 8,
  'Part Number': 'NS-63',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=NS63&type=p&search=s'},
 {'No': 9,
  'Part Number': 'NS-68',
  'Url': 'htt

In [5]:
df_correct = pd.DataFrame(list_dict_correct)
df_correct.to_excel('./test_part_number.xlsx', index=False)

df_correct

,No,Part Number,Url
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...
1,2,CCR-6,https://www.standardbrand.com/en/ecatalog?part...
2,3,CCR-7,https://www.standardbrand.com/en/ecatalog?part...
3,4,DS-2282,https://www.standardbrand.com/en/ecatalog?part...
4,5,LS-312,https://www.standardbrand.com/en/ecatalog?part...
...,...,...,...
111,112,SLS-219,https://www.standardbrand.com/en/ecatalog?part...
112,113,SLS-321,https://www.standardbrand.com/en/ecatalog?part...
113,114,SLS-349,https://www.standardbrand.com/en/ecatalog?part...
114,115,SLS-385,https://www.standardbrand.com/en/ecatalog?part...


In [6]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Request_Url    https://ecatalog.smpcorp.com/V2/STD/api/part/p...
 Name: 0, dtype: object]

In [7]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_number_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.standardbrand.com/en/ecatalog?part...,https://ecatalog.smpcorp.com/V2/STD/api/part/p...
